## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-12-21-46-26 +0000,nyt,How Trump’s Chicago Immigration Crackdown Esca...,https://www.nytimes.com/2025/10/12/us/chicago-...
1,2025-10-12-21-46-04 +0000,nyt,Trump Has Not Ruled Out Invoking Insurrection ...,https://www.nytimes.com/2025/10/12/us/politics...
2,2025-10-12-21-42-29 +0000,nypost,Zelensky suggests Tomahawk missiles could make...,https://nypost.com/2025/10/12/us-news/zelensky...
3,2025-10-12-21-40-54 +0000,nyt,Helena Moreno Is Elected Mayor of New Orleans,https://www.nytimes.com/2025/10/12/us/new-orle...
4,2025-10-12-21-22-53 +0000,nypost,Trump shares update on Barron’s progress at NY...,https://nypost.com/2025/10/12/us-news/trump-sh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,22
42,china,10
76,deal,9
113,gaza,9
79,hamas,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-10-12-20-55-49 +0000,nyt,"Trump leaves for a visit to Israel, and will l...",https://www.nytimes.com/live/2025/10/12/us/tru...,71
97,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,65
152,2025-10-12-04-37-28 +0000,bbc,Israelis praise Trump at huge rally ahead of e...,https://www.bbc.com/news/articles/c75q5qng0rgo...,58
24,2025-10-12-20-10-31 +0000,nypost,Jake Sullivan begrudgingly credits Trump for I...,https://nypost.com/2025/10/12/us-news/jake-sul...,56
165,2025-10-12-02-04-52 +0000,bbc,Trump directs Pentagon to 'use all available f...,https://www.bbc.com/news/articles/ce808gvp56mo...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,71,2025-10-12-20-55-49 +0000,nyt,"Trump leaves for a visit to Israel, and will l...",https://www.nytimes.com/live/2025/10/12/us/tru...
53,42,2025-10-12-17-31-50 +0000,nypost,Father of Israeli-American hostage assumed mur...,https://nypost.com/2025/10/12/world-news/hosta...
144,37,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
39,32,2025-10-12-19-09-17 +0000,nypost,Ian Watkins’ ex shocked pedophile rocker wasn’...,https://nypost.com/2025/10/12/us-news/ian-watk...
160,28,2025-10-12-03-00-00 +0000,wsj,As the government shutdown drags on with no en...,https://www.wsj.com/politics/policy/government...
162,28,2025-10-12-02-45-06 +0000,latimes,Former schools chief Austin Beutner plans to c...,https://www.latimes.com/california/story/2025-...
103,28,2025-10-12-12-00-00 +0000,cbc,Canada-U.S. trade talks moving ‘in the right d...,https://www.cbc.ca/news/politics/canada-tariff...
2,26,2025-10-12-21-42-29 +0000,nypost,Zelensky suggests Tomahawk missiles could make...,https://nypost.com/2025/10/12/us-news/zelensky...
98,26,2025-10-12-12-33-11 +0000,cbc,Afghanistan says it killed 58 Pakistani soldie...,https://www.cbc.ca/news/world/afghanistan-kill...
14,25,2025-10-12-20-43-46 +0000,nypost,Hempstead Town officials ‘secretly’ plotted to...,https://nypost.com/2025/10/12/us-news/hempstea...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
